In [49]:
import os
from pyciemss.PetriNetODE.interfaces import (
    load_and_sample_petri_model,
    load_and_calibrate_and_sample_petri_model,
    load_petri_model,
    setup_petri_model,
    sample
)
from typing import Iterable

In [3]:
cd ../..

/Users/zuck016/Projects/Proposals/ASKEM/build/pyciemss


In [2]:
DEMO_PATH = "notebook/integration_demo/"
ASKENET_PATH = "https://raw.githubusercontent.com/DARPA-ASKEM/Model-Representations/main/petrinet/examples/sir_typed.json"

## load_and_sample_petri_model

In [82]:
num_samples = 3
timepoints = [0.5, 1.0, 2.0, 3.0, 4.0]

# Run sampling w/o an intervention
samples = load_and_sample_petri_model(ASKENET_PATH, num_samples, timepoints=timepoints)

# Save results
samples.to_csv(os.path.join(DEMO_PATH, "results_petri/sample_results.csv"), index=False)

In [22]:
from pyciemss.PetriNetODE.interfaces import load_petri_model, intervene
from pyciemss.PetriNetODE.base import get_name
raw_model = load_petri_model(ASKENET_PATH)
initial_model = setup_petri_model(raw_model, start_time=-1e-10, start_state = {
            get_name(v): v.data["initial_value"] for v in raw_model.G.variables.values()
        }
    )
intervened_model = intervene(initial_model, interventions)
intervened_samples2 = sample(intervened_model, timepoints=timepoints, num_samples=num_samples)
intervened_samples2

{'beta': tensor([0.0263, 0.0247, 0.0258]),
 'gamma': tensor([0.1480, 0.1424, 0.1304]),
 'Infected_sol': tensor([[  0.9410,   0.8854,   3.4494,  22.1137, 129.6330],
         [  0.9428,   0.8889,   3.4701,  22.2437, 130.3026],
         [  0.9490,   0.9006,   3.5332,  22.6397, 132.3341]]),
 'Recovered_sol': tensor([[0.0718, 0.1393, 0.5872, 1.6634, 7.8549],
         [0.0692, 0.1344, 0.5827, 1.6652, 7.8909],
         [0.0635, 0.1238, 0.5754, 1.6774, 8.0069]]),
 'Susceptible_sol': tensor([[999.9871, 999.9749, 996.9626, 977.2221, 863.5111],
         [999.9881, 999.9772, 996.9469, 977.0920, 862.8077],
         [999.9874, 999.9756, 996.8924, 976.6848, 860.6607]])}

In [14]:
# Run sampling w/o an intervention
# intervention is an iterable (e.g. list) of tuples of the form (timepoint: float, parameter: str, value: float)
interventions = [(1.1, "beta", 1.0), (2.1, "gamma", 0.1), (1.3, "beta", 2.0), (1.4, "gamma", 0.3)]

intervened_samples = load_and_sample_petri_model(
    ASKENET_PATH, num_samples, timepoints=timepoints, interventions=interventions
)


# Save results
intervened_samples.to_csv(
    os.path.join(DEMO_PATH, "results_petri/sample_results_w_interventions.csv"),
    index=False,
)

In [25]:
list(np.array([1,.3, .4]))

[1.0, 0.3, 0.4]

In [14]:
# Run sampling w/o an intervention
# intervention is an iterable (e.g. list) of tuples of the form (timepoint: float, parameter: str, value: float)
interventions = [(1.1, "beta", 1.0), (2.1, "gamma", 0.1), (1.3, "beta", 2.0), (1.4, "gamma", 0.3)]

intervened_samples = load_petri_model(
    ASKENET_PATH
)
setup_petri_model(


# Save results
intervened_samples.to_csv(
    os.path.join(DEMO_PATH, "results_petri/sample_results_w_interventions.csv"),
    index=False,
)

In [26]:
np.tile(np.array(range(num_timepoints)), num_samples),

(array([0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4]),)

In [27]:
np.repeat(np.array(range(num_samples)), num_timepoints),

(array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2]),)

In [45]:
np.repeat(np.transpose(np.array([[3,2,2]])), 3, axis=1)

array([[3, 3, 3],
       [2, 2, 2],
       [2, 2, 2]])

In [86]:
sorted(interventions)

[(1.1, 'beta', 1.0),
 (1.3, 'beta', 2.0),
 (1.4, 'gamma', 0.3),
 (2.1, 'gamma', 0.1)]

In [11]:
import numpy as np
v = [0.02441, 0.026181, 0.028274]
num_timepoints=5
np.repeat(v, num_timepoints)

array([0.02441 , 0.02441 , 0.02441 , 0.02441 , 0.02441 , 0.026181,
       0.026181, 0.026181, 0.026181, 0.026181, 0.028274, 0.028274,
       0.028274, 0.028274, 0.028274])

In [47]:
initial_values= {'beta': [0.2, 0.25, 0.15], 'gamma': [0.3, 0.25, 0.35]}

In [87]:
def intervention_to_interval(interventions, initial_values):
    param_dict = {param: [dict(start=0, end=np.inf, expected_value=initial_values[param])] for param in initial_values}
    for i, (start, param, value) in enumerate(sorted(interventions)):
        param_dict[param][-1]['end'] = start
        param_dict[param].append( dict(start=start, end=np.inf, expected_value=[value]*len(initial_values[param])))
    return {
        k: sorted(v, key=lambda x: x['start'])
        for k, v in param_dict.items()
    }        
param_dict = intervention_to_interval(interventions, initial_values)
param_dict

{'beta': [{'start': 0, 'end': 1.1, 'expected_value': [0.2, 0.25, 0.15]},
  {'start': 1.1, 'end': 1.3, 'expected_value': [1.0, 1.0, 1.0]},
  {'start': 1.3, 'end': inf, 'expected_value': [2.0, 2.0, 2.0]}],
 'gamma': [{'start': 0, 'end': 1.4, 'expected_value': [0.3, 0.25, 0.35]},
  {'start': 1.4, 'end': 2.1, 'expected_value': [0.3, 0.3, 0.3]},
  {'start': 2.1, 'end': inf, 'expected_value': [0.1, 0.1, 0.1]}]}

In [81]:
interventions, timepoints, initial_values

([(1.1, 'beta', 1.0),
  (2.1, 'gamma', 0.1),
  (1.3, 'beta', 2.0),
  (1.4, 'gamma', 0.3)],
 [2, 5, 12, 22, 25, 35],
 {'beta': [0.2, 0.25, 0.15], 'gamma': [0.3, 0.25, 0.35]})

In [73]:
param_dict = intervention_to_interval(interventions, initial_values)
param_dict

{'beta': [{'start': 0, 'end': 1.1, 'expected_value': [0.2, 0.25, 0.15]},
  {'start': 1.1, 'end': 1.3, 'expected_value': [1.0, 1.0, 1.0]},
  {'start': 1.3, 'end': inf, 'expected_value': [2.0, 2.0, 2.0]}],
 'gamma': [{'start': 0, 'end': 2.1, 'expected_value': [0.3, 0.25, 0.35]},
  {'start': 1.4, 'end': inf, 'expected_value': [0.3, 0.3, 0.3]},
  {'start': 2.1, 'end': 1.4, 'expected_value': [0.1, 0.1, 0.1]}]}

In [ ]:
def get_intervened_parameter_values(interventions: dict, timepoints: Iterable[float], initial_values: dict) -> dict:
    """Generate a len(timepoints)*len(samples) array of parameter values for each parameter in interventions.

    :param interventions: dict of interventions keyed by parameter name with values (start, value)
    :param timepoints: iterable of timepoints
    :param initial_values: dict of initial values keyed by parameter name with the unintervened parameter value
    """
    param_dict = intervention_to_interval(interventions, initial_values)
    result = {}
    for param, intervals in param_dict.items():
        result[param] = []
        for interval in intervals:
            for i, sample in enumerate(interval['expected_value']):
                result[param].extend([sample] * sum((interval['start'] <= t < interval['end'] for t in timepoints)))
    return result

d = get_intervened_parameter_values(interventions, timepoints, initial_values)
d

In [ ]:
def test_intervention_values(expected_param_values: dict, actual_param_values: dict, idx2timepoint):
    """Check that time-dependent parameter values are set correctly."""
    for param, expected_intervention_assignments in expected_param_values.items():
        for d in expected_intervention_assignments:
            for actual_timepoint, actual_value in actual_param_values[param]:
                if d['start'] < actual_timepoint and d['end'] > actual_timepoint:
                    assert d['expected_value'] == actual_value

Given a set of values at particular intervals,  a set of initial samples and a set of timepoints, generate a list of timepoints where the parameter has the same value at each timepoint in the interval.


In [67]:
import bisect

# initial samples in the form of (start_time, end_time, parameter_value)
samples = [(0, 10, 'a'), (10, 20, 'b'), (20, 30, 'c')]

# timepoints
timepoints = [2, 5, 12, 22, 25, 35]

# transform samples into intervals and parameter values
intervals = [(start, end) for start, end, _ in samples]
values = [val for _, _, val in samples]

# generate list of parameter values at each timepoint
result = []
for t in timepoints:
    i = bisect.bisect(intervals, (t,)) - 1
    if 0 <= i < len(intervals) and intervals[i][0] <= t < intervals[i][1]:
        result.append((t, values[i]))
    else:
        result.append((t, None))

print(result)


[(2, 'a'), (5, 'a'), (12, 'b'), (22, 'c'), (25, 'c'), (35, None)]


In [68]:
import bisect

# initial samples in the form of (start_time, end_time, [parameter_values])
samples = [(0, 10, ['a', 'b']), (10, 20, ['c', 'd']), (20, 30, ['e', 'f'])]

# timepoints
timepoints = [2, 5, 12, 22, 25, 35]

# transform samples into intervals and parameter values
intervals = [(start, end) for start, end, _ in samples]
values_sets = [values for _, _, values in samples]

# generate list of parameter values at each timepoint
result = []
for values in zip(*values_sets):
    for t in timepoints:
        i = bisect.bisect(intervals, (t,)) - 1
        if 0 <= i < len(intervals) and intervals[i][0] <= t < intervals[i][1]:
            result.append((t, values[i]))
        else:
            result.append((t, None))

print(result)


[(2, 'a'), (5, 'a'), (12, 'c'), (22, 'e'), (25, 'e'), (35, None), (2, 'b'), (5, 'b'), (12, 'd'), (22, 'f'), (25, 'f'), (35, None)]


In [88]:
import bisect

def assign_parameters_to_timepoints(interventions: dict, timepoints: Iterable[float], initial_values: dict) -> list:
    # transform samples into intervals and parameter values
    param_interval_dict = intervention_to_interval(interventions, initial_values)
    result = {}
    for param, interval_dict in param_interval_dict.items():
        intervals = [(d['start'], d['end']) for d in interval_dict]
        values_sets = [d['expected_value'] for  d in interval_dict]

        # generate list of parameter values at each timepoint
        result[param] = []
        for values in zip(*values_sets):
            for t in timepoints:
                i = bisect.bisect(intervals, (t,)) - 1
                if 0 <= i < len(intervals) and intervals[i][0] <= t < intervals[i][1]:
                    result[param].append((t, values[i]))
                else:
                    result[param].append((t, None))
    return result

In [89]:
interventions, timepoints, initial_values

([(1.1, 'beta', 1.0),
  (2.1, 'gamma', 0.1),
  (1.3, 'beta', 2.0),
  (1.4, 'gamma', 0.3)],
 [0.5, 1.0, 2.0, 3.0, 4.0],
 {'beta': [0.2, 0.25, 0.15], 'gamma': [0.3, 0.25, 0.35]})

In [90]:
assign_parameters_to_timepoints(interventions, timepoints, initial_values)

{'beta': [(0.5, 0.2),
  (1.0, 0.2),
  (2.0, 2.0),
  (3.0, 2.0),
  (4.0, 2.0),
  (0.5, 0.25),
  (1.0, 0.25),
  (2.0, 2.0),
  (3.0, 2.0),
  (4.0, 2.0),
  (0.5, 0.15),
  (1.0, 0.15),
  (2.0, 2.0),
  (3.0, 2.0),
  (4.0, 2.0)],
 'gamma': [(0.5, 0.3),
  (1.0, 0.3),
  (2.0, 0.3),
  (3.0, 0.1),
  (4.0, 0.1),
  (0.5, 0.25),
  (1.0, 0.25),
  (2.0, 0.3),
  (3.0, 0.1),
  (4.0, 0.1),
  (0.5, 0.35),
  (1.0, 0.35),
  (2.0, 0.3),
  (3.0, 0.1),
  (4.0, 0.1)]}

## load_and_calibrate_and_sample_petri_model

In [7]:
data_path = os.path.join(DEMO_PATH, "data.csv")
num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0]

# Run the calibration and sampling
calibrated_samples = load_and_calibrate_and_sample_petri_model(
    ASKENET_PATH,
    data_path,
    num_samples,
    timepoints=timepoints,
    verbose=True,
)

# Save results
calibrated_samples.to_csv(
    os.path.join(DEMO_PATH, "results_petri/calibrated_sample_results.csv"), index=False
)

iteration 0: loss = 37.94811773300171
iteration 25: loss = 34.9170196056366
iteration 50: loss = 32.69080424308777
iteration 75: loss = 32.41730046272278
iteration 100: loss = 32.44331431388855
iteration 125: loss = 32.692076683044434
iteration 150: loss = 32.61838483810425
iteration 175: loss = 32.56607246398926
iteration 200: loss = 33.014737367630005
iteration 225: loss = 32.58268332481384
iteration 250: loss = 32.89216685295105
iteration 275: loss = 31.961626768112183
iteration 300: loss = 32.6565682888031
